# Consequence of Failure Analysis

## Proximity Analysis

In [2]:
import arcpy
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import os
from datetime import datetime
import math

In [3]:
# Set the workspace
arcpy.env.workspace = r"C:\Users\ggarcia\OneDrive - Abonmarche\GIS Projects\2023\23-0304 Grand Haven CDSMI\ArcGIS Pro\GH Risk Analysis\Round3Analysis.gdb"
arcpy.env.overwriteOutput = True

In [4]:
# water main feature class to dataframe
water_main_fc = "Round3WaterMain"
water_main_df = pd.DataFrame.spatial.from_featureclass(water_main_fc)
# water_main_df.head()

In [5]:
# drop all columns except OBJECTID, FACILITYID, and DIAMETER
water_main_df = water_main_df[['OBJECTID', 'FACILITYID', 'DIAMETER']]
water_main_df.head()

,OBJECTID,FACILITYID,DIAMETER
0,1,1,8.0
1,2,2,6.0
2,3,3,6.0
3,4,4,4.0
4,5,5,12.0


In [6]:
# drop all rows with null values
water_main_df.dropna(inplace=True)

### Iterate Near Analysis

In [ ]:
# List of feature classes
feature_classes = ["Buildings", "Major_Intersection", "Major_Road", "Minor_Intersection", "Minor_Road", "Railroad", "ROW", "WaterAreas", "WaterLines"]
# Get the current working directory
dir_path = os.getcwd()
# Dictionary to store the dataframes
dfs = {}

# Iterate over the feature classes
for fc in feature_classes:
    # Create the output path
    output_path = os.path.join(dir_path, "Results", "near_" + fc + ".csv")
    # Near analysis
    near_table = arcpy.GenerateNearTable_analysis(
        in_features=water_main_fc,
        near_features=fc,
        out_table=output_path,
        search_radius="10000 Feet",
        location="NO_LOCATION",
        angle="NO_ANGLE",
        closest="CLOSEST",
        closest_count="0",
        method="PLANAR",
        distance_unit="Feet")

    # Convert near table csv to dataframe
    near_df = pd.read_csv(output_path)

    # remove the .xml, .ini, and .csv.xml files the geoprocess also created
    for file in os.listdir(os.path.join(dir_path, "Results")):
        if file.endswith(".xml") or file.endswith(".ini"):
            os.remove(os.path.join(dir_path, "Results", file))

    # rename the column in the near_df from NEAR_DIST to the name of the feature class
    near_df.rename(columns = {'NEAR_DIST': fc}, inplace = True)
    # drop the columns OBJECTID and NEAR_FID
    near_df = near_df.drop(columns=['NEAR_FID'])

    # Add the dataframe to the dictionary
    dfs[fc] = near_df
# copy the water_main_df to a new dataframe
Near_results_df = water_main_df.copy()

# Merge the dataframes with the Near_results_df removing the IN_FID column each time
for key, value in dfs.items():
    Near_results_df = pd.merge(Near_results_df, value, left_on='OBJECTID', right_on='IN_FID', how='left')
    Near_results_df = Near_results_df.drop(columns=['IN_FID'])

Near_results_df.head()
# save the Near_results_df to a csv file using dir_path
Near_results_df.to_csv(os.path.join(dir_path, "Results", "NearResults.csv"), index=False)

## Affected Customer Analysis

### Affected Service Connections

In [ ]:
# define isolation zones and lateral lines feature classes

isolation_zones_fc = "IsoZone"
lateral_lines_fc = "Round3WaterLats"

# filter the laterals to only domestic lines
domestic_lats = arcpy.management.SelectLayerByAttribute(lateral_lines_fc, "NEW_SELECTION", "LINETYPE <> 'Hydrant'")
# spatial join laterals to isolation zones one to many so there is a row for each lateral in the isolation zone
zones_lats_join = "in_memory\zones_lats_join"

arcpy.analysis.SpatialJoin(
    target_features=isolation_zones_fc,
    join_features=domestic_lats,
    out_feature_class=zones_lats_join,
    join_operation="JOIN_ONE_TO_MANY",
    join_type="KEEP_COMMON",
    match_option="INTERSECT",
)
#summarize the join results

summary_output = os.path.join(dir_path, "Results", "zone_lat_summary" + ".csv")
arcpy.analysis.Statistics(
    in_table=zones_lats_join,
    out_table=summary_output,
    statistics_fields="FACILITYID COUNT",
    case_field="zone",
)
# remove the .xml, .ini, and .csv.xml files the geoprocess also created
for file in os.listdir(os.path.join(dir_path, "Results")):
    if file.endswith(".xml") or file.endswith(".ini"):
        os.remove(os.path.join(dir_path, "Results", file))
# convert the summary table to a dataframe
summary_df = pd.read_csv(summary_output)
summary_df.head()
# add a spatial join to the water main feature class to get the isolation zones into the water mains
main_iso_join = "in_memory\main_iso_join"

arcpy.analysis.SpatialJoin(
    target_features=water_main_fc,
    join_features=isolation_zones_fc,
    out_feature_class=main_iso_join,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="LARGEST_OVERLAP"
)
# Convert the feature class to a dataframe
mains_iso_df = pd.DataFrame.spatial.from_featureclass(main_iso_join)
mains_iso_df.head()
# drop all columns except OBJECTID, FACILITYID, DIAMETER, and zone
mains_iso_df = mains_iso_df[['OBJECTID', 'FACILITYID', 'DIAMETER', 'zone']]
mains_iso_df.head()
#  use the summary df as a key to add a column to the mains_iso_df for affected laterals and fill it with the count of laterals in the isolation zone
mains_iso_df['affected_lats'] = mains_iso_df['zone'].map(summary_df.set_index('zone')['FREQUENCY'])
mains_iso_df.head()
# save the mains_iso_df to a csv using the dir_path
mains_iso_df.to_csv(os.path.join(dir_path, "Results", "mains_affected_lats_by_zones.csv"), index=False)

### Affected Critical Customers

#### Schools/Childcare Steps

In [ ]:
# spatial join merged school and childcare feature class to parcels
parcels_fc = "Parcels"
school_childcare = "School_Childcare"

parcels_school_child_join = "in_memory\parcels_school_child_join"

arcpy.analysis.SpatialJoin(
    target_features=parcels_fc,
    join_features=school_childcare,
    out_feature_class=parcels_school_child_join,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="Keep_Common",
)
# identify lateral lines that intersect the spatial join of parcels and school/childcare using spatial join
lats_School_Child_Join = "in_memory\Lats_School_Child_Join"

arcpy.analysis.SpatialJoin(
    target_features=lateral_lines_fc,
    join_features=parcels_school_child_join,
    out_feature_class=lats_School_Child_Join,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
)
# spatial join the school lateral results to the rest of the laterals by intersect to identify both sides of the service that serve the school/childcare
lats_School_Child_Join_All = "in_memory\Lats_School_Child_Join_All"

arcpy.analysis.SpatialJoin(
    target_features=lateral_lines_fc,
    join_features=lats_School_Child_Join,
    out_feature_class=lats_School_Child_Join_All,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
    match_option="INTERSECT",
)
# dissolve the spatial join results by parcel id to create one line feature for each school/childcare service connection
lats_school_child_dissolve = "in_memory\Lats_School_Child_Dissolve"

arcpy.management.Dissolve(
    in_features=lats_School_Child_Join_All,
    out_feature_class=lats_school_child_dissolve,
    dissolve_field="FinalPIN",
)
# spatial join the dissolved feature class to the water main feature class to identify the mains that serve the school/childcare
Main_School_Child_Join = "in_memory\Main_School_Child_Join"

arcpy.analysis.SpatialJoin(
    target_features=water_main_fc,
    join_features=lats_school_child_dissolve,
    out_feature_class=Main_School_Child_Join,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
)
# turn the feature class into a dataframe
main_school_child_df = pd.DataFrame.spatial.from_featureclass(Main_School_Child_Join)

# drop all columns except OBJECTID, FACILITYID, zone, and PIN
main_school_child_df = main_school_child_df[['OBJECTID', 'FACILITYID', 'FinalPIN']]
# add a column called school_childcare and fill it with 1
main_school_child_df['school_childcare'] = "Connected"
main_school_child_df.head()
# save the dataframes to csv file with directory path variable
main_school_child_df.to_csv(os.path.join(dir_path, "Results", "mains_school_childcare.csv"), index=False)

<Result 'in_memory\\parcels_school_child_join'>

#### Medical Facilities

In [ ]:
# spatial join medical facilities to parcels
parcels_fc = "Parcels"
medical_facilities_fc = "HealthCare"

# store output in memory
medical_parcels = "in_memory/medical_parcels"

arcpy.analysis.SpatialJoin(
    target_features=parcels_fc,
    join_features=medical_facilities_fc,
    out_feature_class=medical_parcels,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="Keep_Common",
)
# identify lateral lines that intersect the spatial join of parcels and medical facilities using spatial join
lats_medical = "in_memory/lats_medical"

arcpy.analysis.SpatialJoin(
    target_features=lateral_lines_fc,
    join_features=medical_parcels,
    out_feature_class=lats_medical,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
)
# spatial join the medical lateral results to the rest of the laterals by intersect to identify both sides of the service that serve the medical facilities
lats_medical_all = "in_memory/lats_medical_all"

arcpy.analysis.SpatialJoin(
    target_features=lateral_lines_fc,
    join_features=lats_medical,
    out_feature_class=lats_medical_all,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
    match_option="INTERSECT",
)
# dissolve the spatial join results by parcel id to create one line feature for each school/childcare service connection
lats_medical_dissolve = "in_memory/lats_medical_dissolve"

arcpy.management.Dissolve(
    in_features=lats_medical_all,
    out_feature_class=lats_medical_dissolve,
    dissolve_field="FinalPIN",
)
# spatial join the dissolved feature class to the water main feature class to identify the mains that serve the school/childcare
main_medical = "in_memory/main_medical"

arcpy.analysis.SpatialJoin(
    target_features=water_main_fc,
    join_features=lats_medical_dissolve,
    out_feature_class=main_medical,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
)
# turn the feature class into a dataframe
main_medical_df = pd.DataFrame.spatial.from_featureclass(main_medical)

# drop all columns except OBJECTID, FACILITYID, and PARCEL_NO
main_medical_df = main_medical_df[['OBJECTID', 'FACILITYID', 'FinalPIN']]
# add a column called medical and fill it with 1
main_medical_df['medical'] = "Connected"
main_medical_df.head()
# save the dataframes to csv file with directory path variable
main_medical_df.to_csv(os.path.join(dir_path, "Results", "mains_medical.csv"), index=False)

<Result 'in_memory\\medical_parcels'>

#### Additional Critical Customers

In [ ]:
# spatial join critical customer facilities to parcels
parcels_fc = "Parcels"
crit_customers_fc = "AdditionalCriticalCustomers"

# store output in memory
critical_cust_parcels = "in_memory/critical_cust_parcels"

arcpy.analysis.SpatialJoin(
    target_features=parcels_fc,
    join_features=crit_customers_fc,
    out_feature_class=critical_cust_parcels,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="Keep_Common",
)
# identify lateral lines that intersect the spatial join of parcels and medical facilities using spatial join
lats_critical_cust = "in_memory/lats_critical_cust"

arcpy.analysis.SpatialJoin(
    target_features=lateral_lines_fc,
    join_features=critical_cust_parcels,
    out_feature_class=lats_critical_cust,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
)
# spatial join the crit cust lateral results to the rest of the laterals by intersect to identify both sides of the service that serve the medical facilities
lats_critical_cust_all = "in_memory/lats_critical_cust_all"

arcpy.analysis.SpatialJoin(
    target_features=lateral_lines_fc,
    join_features=lats_critical_cust,
    out_feature_class=lats_critical_cust_all,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
    match_option="INTERSECT",
)
# dissolve the spatial join results by parcel id to create one line feature for each school/childcare service connection
lats_critical_cust_dissolve = "in_memory/lats_critical_cust_dissolve"

arcpy.management.Dissolve(
    in_features=lats_critical_cust_all,
    out_feature_class=lats_critical_cust_dissolve,
    dissolve_field="FinalPIN",
)
# spatial join the dissolved feature class to the water main feature class to identify the mains that serve the school/childcare
main_critical_cust = "in_memory/main_critical_cust"

arcpy.analysis.SpatialJoin(
    target_features=water_main_fc,
    join_features=lats_critical_cust_dissolve,
    out_feature_class=main_critical_cust,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
)
# turn the feature class into a dataframe
main_critical_cust_df = pd.DataFrame.spatial.from_featureclass(main_critical_cust)

# drop all columns except OBJECTID, FACILITYID, and PARCEL_NO
main_critical_cust_df = main_critical_cust_df[['OBJECTID', 'FACILITYID', 'FinalPIN']]
# add a column called medical and fill it with 1
main_critical_cust_df['criticalcust'] = "Connected"
main_critical_cust_df.head()
# save the dataframes to csv file with directory path variable
main_critical_cust_df.to_csv(os.path.join(dir_path, "Results", "mains_critical_cust.csv"), index=False)

<Result 'in_memory\\critical_cust_parcels'>

## Scoring

### Compile data into one dataframe for calculations

Compile data from saved csv's in case calculations need to be redone based on just one updated parameter section

1. Near_results.csv - distance to proximity features for each main segment
2. mains_affected_lats_by_zones.csv - summary of isolation zones and affected customers per zone for each main segment
3. mains_school_childcare.csv - mains connected to critical school/childcare customer
4. mains_medical.csv - mains connected to critical medical facility customers
5. mains_critical_cust.csv - mains connected to other critical customers


In [ ]:
# Get the current working directory
dir_path = os.getcwd()
# create dataframes from the csv files
file_names = ["NearResults.csv", "mains_affected_lats_by_zones.csv", "mains_school_childcare.csv", "mains_medical.csv", "mains_critical_cust.csv"]

# create each df individually
near_df = pd.read_csv(os.path.join(dir_path, "Results", file_names[0]))
connections_df = pd.read_csv(os.path.join(dir_path, "Results", file_names[1]))
school_childcare_df = pd.read_csv(os.path.join(dir_path, "Results", file_names[2]))
medical_df = pd.read_csv(os.path.join(dir_path, "Results", file_names[3]))
critical_cust_df = pd.read_csv(os.path.join(dir_path, "Results", file_names[4]))
near_df.head()
# Get new water mains data frame after adding a field that calculates the length of the water mains
# Set the workspace
arcpy.env.workspace = r"C:\Users\ggarcia\OneDrive - Abonmarche\GIS Projects\2023\23-0304 Grand Haven CDSMI\ArcGIS Pro\GH Risk Analysis\Round3Analysis.gdb"
arcpy.env.overwriteOutput = True

water_main_fc = "Round3WaterMain"

# export the feature class to an in memory feature class
water_main_fc_in_memory = "in_memory\water_main_fc"

arcpy.conversion.ExportFeatures(water_main_fc, water_main_fc_in_memory)

arcpy.management.CalculateGeometryAttributes(
    in_features=water_main_fc_in_memory,
    geometry_property=[["LENGTH", "LENGTH"]],
    length_unit="FEET_INT"
)

# convert the feature class to a dataframe
water_main_df = pd.DataFrame.spatial.from_featureclass(water_main_fc_in_memory)
# keep columns OBJECTID, FACILITYID, DIAMETER, and LENGTH
water_main_df = water_main_df[['FACILITYID', 'DIAMETER', 'LENGTH']]
water_main_df.head()
# drop all rows with null values
water_main_df.dropna(inplace=True)
# convert the facilityid column in the near_df to a string then merge the near_df with the water_main_df by the FACILITYID column
near_df['FACILITYID'] = near_df['FACILITYID'].astype(str)
# remove the .0 from the FACILITYID column
near_df['FACILITYID'] = near_df['FACILITYID'].str.split('.').str[0]

# drop objectid and diameter from the near_df
near_df = near_df.drop(columns=['OBJECTID', 'DIAMETER'])

# merge the water_main_df and near_df with a join on FACILITYID from the water_main_df and FACILITYID from the near_df
water_main_df['FACILITYID'] = water_main_df['FACILITYID'].astype(str)
near_merged_df = pd.merge(water_main_df, near_df, on='FACILITYID', how='left')
near_merged_df.head()
# convert the facilityid column in the connections_df to a string then merge the connections_df with the water_main_df by the FACILITYID column
connections_df['FACILITYID'] = connections_df['FACILITYID'].astype(str)
# drop decimal and trailing numbers from the FACILITYID column
connections_df['FACILITYID'] = connections_df['FACILITYID'].str.split('.').str[0]

# drop objectid and diameter from the connections_df
connections_df = connections_df.drop(columns=['OBJECTID', 'DIAMETER'])

# merge the water_main_df and connections_df with a join on FACILITYID from the water_main_df and FACILITYID from the connections_df
connections_near_merged_df = pd.merge(near_merged_df, connections_df, on='FACILITYID', how='left')
connections_near_merged_df.head()
# convert the facilityid column in the school_childcare_df to a string then merge the school_childcare_df with the water_main_df by the FACILITYID column
school_childcare_df['FACILITYID'] = school_childcare_df['FACILITYID'].astype(str)
# drop decimal and trailing numbers from the FACILITYID column
school_childcare_df['FACILITYID'] = school_childcare_df['FACILITYID'].str.split('.').str[0]

# drop objectid and diameter from the school_childcare_df
school_childcare_df = school_childcare_df.drop(columns=['OBJECTID', 'FinalPIN'])

# merge the water_main_df and school_childcare_df with a join on FACILITYID from the water_main_df and FACILITYID from the school_childcare_df
school_connections_near_merged_df = pd.merge(connections_near_merged_df, school_childcare_df, on='FACILITYID', how='left')
school_connections_near_merged_df.head()
# convert the facilityid column in the medical_df to a string then merge the medical_df with the water_main_df by the FACILITYID column
medical_df['FACILITYID'] = medical_df['FACILITYID'].astype(str)
# drop decimal and trailing numbers from the FACILITYID column
medical_df['FACILITYID'] = medical_df['FACILITYID'].str.split('.').str[0]

# drop objectid and parcel_no from the medical_df
medical_df = medical_df.drop(columns=['OBJECTID', 'FinalPIN'])

# merge the water_main_df and medical_df with a join on FACILITYID from the water_main_df and FACILITYID from the medical_df
final_df = pd.merge(school_connections_near_merged_df, medical_df, on='FACILITYID', how='left')
final_df.head()
# convert the facilityid column in the critical_cust_df to a string then merge the critical_cust_df with the water_main_df by the FACILITYID column
critical_cust_df['FACILITYID'] = critical_cust_df['FACILITYID'].astype(str)
# drop decimal and trailing numbers from the FACILITYID column
critical_cust_df['FACILITYID'] = critical_cust_df['FACILITYID'].str.split('.').str[0]

# drop objectid and parcel_no from the critical_cust_df
critical_cust_df = critical_cust_df.drop(columns=['OBJECTID', 'FinalPIN'])

# merge the water_main_df and critical_cust_df with a join on FACILITYID from the water_main_df and FACILITYID from the critical_cust_df
final_df = pd.merge(final_df, critical_cust_df, on='FACILITYID', how='left')
final_df.head()
# save the final_df to a csv file using dir_path
output_path = os.path.join(dir_path, "Results", "Combined_Parameters.csv")
final_df.to_csv(output_path, index=False)

### Isolation Zone analysis

In [ ]:
# final_df from the csv file
final_df = pd.read_csv(output_path)
# for all school_childcare column values of connected store the zone value, if school_childcare is null and the zone is in the list of zones with connected school_childcare, set the value to zone
zones = []

# Iterate over the rows in the dataframe
for index, row in final_df.iterrows():
    # If the 'school_childcare' value is "Connected", add the zone to the list
    if row['school_childcare'] == "Connected":
        zones.append(row['zone'])

# Convert the list to a set for faster lookup
zones = set(zones)

# Iterate over the rows in the dataframe again
for index, row in final_df.iterrows():
    # If the 'school_childcare' value is null and the zone is in the list, set the 'school_childcare' value to "Zone"
    if pd.isnull(row['school_childcare']) and row['zone'] in zones:
        final_df.loc[index, 'school_childcare'] = "Zone"

# for all medical column values of connected store the zone value, if medical is null and the zone is in the list of zones with connected medical, set the value to zone
zones = []

# Iterate over the rows in the dataframe
for index, row in final_df.iterrows():
    # If the 'medical' value is "Connected", add the zone to the list
    if row['medical'] == "Connected":
        zones.append(row['zone'])

# Convert the list to a set for faster lookup
zones = set(zones)

# Iterate over the rows in the dataframe again
for index, row in final_df.iterrows():
    # If the 'medical' value is null and the zone is in the list, set the 'medical' value to "Zone"
    if pd.isnull(row['medical']) and row['zone'] in zones:
        final_df.loc[index, 'medical'] = "Zone"
# for all criticalcust column values of connected store the zone value, if criticalcust is null and the zone is in the list of zones with connected criticalcust, set the value to zone
zones = []

# Iterate over the rows in the dataframe
for index, row in final_df.iterrows():
    # If the 'criticalcust' value is "Connected", add the zone to the list
    if row['criticalcust'] == "Connected":
        zones.append(row['zone'])

# Convert the list to a set for faster lookup
zones = set(zones)

# Iterate over the rows in the dataframe again
for index, row in final_df.iterrows():
    # If the 'criticalcust' value is null and the zone is in the list, set the 'criticalcust' value to "Zone"
    if pd.isnull(row['criticalcust']) and row['zone'] in zones:
        final_df.loc[index, 'criticalcust'] = "Zone"
final_df.head()
# save the final_df to a csv file using dir_path
output_path = os.path.join(dir_path, "Results", "Combined_Parameters_zone_check.csv")
final_df.to_csv(output_path, index=False)

### Score assignment

In [ ]:
# drop the zone column from the final_df
final_df = final_df.drop(columns=['zone'])

# add new columns with the specified names and fill them with 0
final_df['Railroad_score'] = 0
final_df['WaterBodies_score'] = 0
final_df['Buildings_score'] = 0
final_df['Roadway_score'] = 0
final_df['affected_lats_score'] = 0
final_df['school_childcare_score'] = 0
final_df['medical_score'] = 0
final_df['critical_cust_score'] = 0
final_df['DIAMETER_score'] = 0

final_df.head()
def score_diameter(diameter):
    if diameter < 4:
        return 1
    elif 4 <= diameter <= 8:
        return 4
    elif 8 < diameter <= 16:
        return 7
    elif diameter >= 16:
        return 10

# Apply the function to the 'DIAMETER' column to calculate the 'DIAMETER_score'
final_df['DIAMETER_score'] = final_df['DIAMETER'].apply(score_diameter)
final_df.head()
def score_railroad(railroad):
    if railroad == 0:
        return 10
    elif 0 < railroad <= 10:
        return 9
    elif 10 < railroad <= 50:
        return 7
    elif 50 < railroad <= 100:
        return 5
    elif railroad > 100:
        return 0

# Apply the function to the 'Railroad' column to calculate the 'Railroad_score'
final_df['Railroad_score'] = final_df['Railroad'].apply(score_railroad)
final_df.head()
def score_waterbodies(WaterAreas, WaterLines):
    waterbodies = min(WaterAreas, WaterLines)
    if waterbodies == 0:
        return 10
    elif 0 < waterbodies <= 10:
        return 9
    elif 10 < waterbodies <= 50:
        return 7
    elif 50 < waterbodies <= 100:
        return 5
    elif waterbodies > 100:
        return 0

# Apply the function to the 'WaterAreas' and 'WaterLines' columns to calculate the 'WaterBodies_score'
final_df['WaterBodies_score'] = final_df.apply(lambda row: score_waterbodies(row['WaterAreas'], row['WaterLines']), axis=1)
final_df.head()
def score_buildings(buildings):
    if 0 <= buildings <= 5:
        return 10
    elif 5 < buildings <= 20:
        return 5
    elif buildings > 20:
        return 0

# Apply the function to the 'Buildings' column to calculate the 'Buildings_score'
final_df['Buildings_score'] = final_df['Buildings'].apply(score_buildings)
final_df.head()
def score_affected_lats(affected_lats):
    if pd.isnull(affected_lats) or affected_lats == 0:
        return 0
    elif affected_lats > 50:
        return 10
    elif 31 <= affected_lats <= 50:
        return 8
    elif 11 <= affected_lats <= 30:
        return 5
    elif 1 <= affected_lats <= 10:
        return 1

# Apply the function to the 'affected_lats' column to calculate the 'affected_lats_score'
final_df['affected_lats_score'] = final_df['affected_lats'].apply(score_affected_lats)
final_df.head()
def score_school_childcare(school_childcare):
    if school_childcare == "Connected":
        return 10
    elif school_childcare == "Zone":
        return 8
    else:
        return 0

# Apply the function to the 'school_childcare' column to calculate the 'school_childcare_score'
final_df['school_childcare_score'] = final_df['school_childcare'].apply(score_school_childcare)
final_df.head()
def score_medical(medical):
    if medical == "Connected":
        return 10
    elif medical == "Zone":
        return 8
    else:
        return 0

# Apply the function to the 'medical' column to calculate the 'medical_score'
final_df['medical_score'] = final_df['medical'].apply(score_medical)
final_df.head()
def score_critical_cust(criticalcust):
    if criticalcust == "Connected":
        return 10
    elif criticalcust == "Zone":
        return 8
    else:
        return 0
    
# Apply the function to the 'criticalcust' column to calculate the 'critical_cust_score'
final_df['critical_cust_score'] = final_df['criticalcust'].apply(score_critical_cust)
final_df.head()
def score_roadway(row):
    if row['Major_Intersection'] == 0:
        return 10
    elif row['Major_Road'] == 0:
        return 9
    elif row['Minor_Intersection'] == 0:
        return 7
    elif row['Minor_Road'] == 0:
        return 6
    elif row['ROW'] == 0 and row['Major_Road'] < row['Minor_Road']:
        return 3
    elif row['ROW'] == 0 and row['Major_Road'] >= row['Minor_Road']:
        return 2
    else:
        return 0

# Apply the function to the rows in the dataframe to calculate the 'Roadway_score'
final_df['Roadway_score'] = final_df.apply(score_roadway, axis=1)
final_df.head()
# save the final_df to a csv file using dir_path
output_path = os.path.join(dir_path, "Results", "Scored_Parameters.csv")
final_df.to_csv(output_path, index=False)

,FACILITYID,DIAMETER,LENGTH,Buildings,Major_Intersection,Major_Road,Minor_Intersection,Minor_Road,Railroad,ROW,...,criticalcust,Railroad_score,WaterBodies_score,Buildings_score,Roadway_score,affected_lats_score,school_childcare_score,medical_score,critical_cust_score,DIAMETER_score
0,1,8.0,67.320030,43.911240,0.000000,0.000000,168.287292,0.000000,6489.103901,0.0,...,NaN,0,0,0,0,0,0,0,0,0
1,2,6.0,353.007987,80.060164,174.284811,195.279288,24.736512,1.345787,3080.661324,0.0,...,NaN,0,0,0,0,0,0,0,0,0
2,3,6.0,29.649311,43.100158,322.949458,299.657014,78.803958,0.000000,5462.586977,0.0,...,NaN,0,0,0,0,0,0,0,0,0
3,4,4.0,942.596550,32.930425,0.000000,0.000000,0.000000,0.000000,340.826979,0.0,...,NaN,0,0,0,0,0,0,0,0,0
4,5,12.0,1722.953951,120.053457,939.162354,891.284866,623.562655,33.950529,4413.770733,0.0,...,NaN,0,0,0,0,0,0,0,0,0


### Final Score

In [ ]:
# load dataframe from csv in case we want to adjust the calculation of scores we already created and add a COF column
final_df = pd.read_csv(output_path)
final_df.head()
# remove any rows where length is null
final_df = final_df[final_df['LENGTH'].notna()]
# calculate the final COF and use math.ceil to round UP to the next whole number

def calculate_final_score(row):
    return math.ceil((row['DIAMETER_score'] * 0.1 +
                  row['Railroad_score'] * 0.1 +
                  row['Roadway_score'] * 0.15 +
                  row['Buildings_score'] * 0.1 +
                  row['affected_lats_score'] * 0.2 +
                  row['WaterBodies_score'] * 0.2 +
                  row['medical_score'] * 0.05 +
                  row['school_childcare_score'] * 0.05 +
                  row['critical_cust_score'] * 0.05))

# Apply the function to the rows in the dataframe to calculate the 'final_score'
final_df['COF'] = final_df.apply(calculate_final_score, axis=1)
final_df.head()
# add a column to normalize the COF to a 1-10 scale with a formula of (10/max cof) * cof
final_df['COF_normalized'] = (10 / final_df['COF'].max()) * final_df['COF']
# round up to the ceiling
final_df['COF_normalized'] = final_df['COF_normalized'].apply(math.ceil)
final_df.head()
# Min-Max normalization to scale between 1 and 10
min_val = final_df['COF'].min()
max_val = final_df['COF'].max()
final_df['COF_normalized'] = ((final_df['COF'] - min_val) / (max_val - min_val) * 9) + 1
# round up to the ceiling
final_df['COF_normalized'] = final_df['COF_normalized'].apply(math.ceil)
final_df.head()
# add column for adjusted_COF and fill it with the value of COF
final_df['adjusted_COF'] = final_df['COF_normalized']

# read the water_adjustments.csv file into a dataframe, use the FACILITYID column to change matching FACILITYID rows in the final_df and change the adjusted_COF to 10
water_adjustments = pd.read_csv(os.path.join(dir_path, "Results", "water_cof_adjustments.csv"))

# iterate over the rows in the water_adjustments dataframe
for index, row in water_adjustments.iterrows():
    # set the adjusted_COF to 10 for the rows in the final_df that match the FACILITYID in the water_adjustments dataframe
    final_df.loc[final_df['FACILITYID'] == row['FACILITYID'], 'adjusted_COF'] = 10
final_df.head()
# save the final_df to a csv file using dir_path
output_path = os.path.join(dir_path, "Results", "Final_COF.csv")
final_df.to_csv(output_path, index=False)

,FACILITYID,DIAMETER,LENGTH,Buildings,Major_Intersection,Major_Road,Minor_Intersection,Minor_Road,Railroad,ROW,...,criticalcust,Railroad_score,WaterBodies_score,Buildings_score,Roadway_score,affected_lats_score,school_childcare_score,medical_score,critical_cust_score,DIAMETER_score
0,1,8.0,67.320030,43.911240,0.000000,0.000000,168.287292,0.000000,6489.103901,0.0,...,NaN,0,0,0,10,5,0,0,0,4
1,2,6.0,353.007987,80.060164,174.284811,195.279288,24.736512,1.345787,3080.661324,0.0,...,NaN,0,0,0,2,1,0,0,0,4
2,3,6.0,29.649311,43.100158,322.949458,299.657014,78.803958,0.000000,5462.586977,0.0,...,NaN,0,0,0,6,0,8,8,0,4
3,4,4.0,942.596550,32.930425,0.000000,0.000000,0.000000,0.000000,340.826979,0.0,...,NaN,0,0,0,10,5,0,0,0,4
4,5,12.0,1722.953951,120.053457,939.162354,891.284866,623.562655,33.950529,4413.770733,0.0,...,NaN,0,0,0,2,1,0,0,0,7
